In [21]:
from astropy.io import fits
from astropy import wcs

import numpy as np
from ants import cvMe

conv = cvMe.convert()

def hms2deg(self,ra_hms):

    ra = string.split(ra_hms, ':')

    hh = float(ra[0])*15
    mm = (float(ra[1])/60)*15
    ss = (float(ra[2])/3600)*15

    raDeg = hh+mm+ss

    return raDeg

# DMS -> degrees
def dms2deg(self,dec_dms):
    dec = string.split(dec_dms, ':')

    dd = abs(float(dec[0]))
    mm = float(dec[1])/60
    ss = float(dec[2])/3600

    if float(dec[0])>= 0:
        decDeg = dd+mm+ss
    else:
        decDeg = -(dd+mm+ss)

    return decDeg


home = '/home/'
#home = '/Users/'

rootDir = home+'maccagni/Projects/MFS/FornaxA/MUSE/'
cubeDir = rootDir+'cubes/'

GAL = 'FornaxE'
RUN = '_5'

resDir= cubeDir+'results/'+GAL+RUN+'/'

inTable = resDir+GAL+'_table.fits'
ppXFTable = resDir+GAL+'_ppxf.fits'
gandalfTable = resDir+GAL+'_gandalf_BIN.fits'
resOut = resDir+'mymaps/'

# these are arbitrary but must correspond, I choose the centre of the galaxy(usually offset with fov)
crPix1=200  
crPix2=124
crVal1='03:22:41.711'
crVal2='-37:12:29.26'


output = resOut+'StarFlux.fits'
outputV = resOut+'StarVel.fits'
outputSigma = resOut+'StarSigma.fits'
outputH3 = resOut+'StarH3.fits'
outputH4 = resOut+'StarH4.fits'
outputLambdaR = resOut+'StarLambdaR.fits'

ra = conv.hms2deg(crVal1)
dec = conv.dms2deg(crVal2)




print '''\t+---------+\n\t Inputs loaded\n\t+---------+''' 

	+---------+
	 Inputs loaded
	+---------+


In [22]:
#----------------------#
# From Fits Table to FITS Image - PPXF
#----------------------#

diffusion = 1e-5

tab = fits.open(inTable)

head = tab[0].header
headTab = tab[1].header
dataTab = tab[1].data


tabPP = fits.open(ppXFTable)

headPP = tabPP[0].header
headTabPP = tabPP[1].header
dataTabPP = tabPP[1].data

head['CRPIX1'] = crPix1
head['CRPIX2'] = crPix2

xMin = np.min(dataTab['X'])
xMax = np.max(dataTab['X'])

shapeX = (xMax-xMin)/head['PIXSIZE']

yMin = np.min(dataTab['Y'])
yMax = np.max(dataTab['Y'])

shapeY = (yMax-yMin)/head['PIXSIZE']


xAxis = (np.linspace(1, shapeX, shapeX) - head['CRPIX1']) *head['PIXSIZE']
yAxis = (np.linspace(1, shapeY, shapeY) - head['CRPIX2']) *head['PIXSIZE']

xIM, yIM = np.meshgrid(xAxis, yAxis)


data = np.zeros([yAxis.shape[0],xAxis.shape[0]])
data[data==0.0] = np.nan

vel = np.zeros([yAxis.shape[0],xAxis.shape[0]])
vel[vel==0.0] = np.nan

sigma = np.zeros([yAxis.shape[0],xAxis.shape[0]])
sigma[sigma==0.0] = np.nan

h3 = np.zeros([yAxis.shape[0],xAxis.shape[0]])
h3[h3==0.0] = np.nan

h4 = np.zeros([yAxis.shape[0],xAxis.shape[0]])
h4[h4==0.0] = np.nan

lambdaR = np.zeros([yAxis.shape[0],xAxis.shape[0]])
lambdaR[lambdaR==0.0] = np.nan

for i in xrange(0,dataTab['ID'].shape[0]):

    indexX = ((xAxis < (np.round(dataTab['X'][i],1)+diffusion)) & 
              ((np.round(dataTab['X'][i],1)-diffusion) < xAxis))
    indexY = ((yAxis < (np.round(dataTab['Y'][i],1)+diffusion)) & 
              ((np.round(dataTab['Y'][i],1)-diffusion) < yAxis))    
    
    indexPPXF = dataTabPP['BIN_ID'] == dataTab['BIN_ID'][i]
    
    if np.sum(indexPPXF)>0:
        
        vel[indexX,indexY] = dataTabPP['V'][indexPPXF]
        sigma[indexX,indexY] = dataTabPP['SIGMA'][indexPPXF]
        h3[indexX,indexY] = dataTabPP['H3'][indexPPXF]
        h4[indexX,indexY] = dataTabPP['H4'][indexPPXF]
        h4[indexX,indexY] = dataTabPP['H4'][indexPPXF]
        lambdaR[indexX,indexY] = dataTabPP['LAMBDA_R'][indexPPXF]
        
    data[indexX,indexY] = dataTab['FLUX'][i]

fits.writeto(output,data,head,overwrite=True)
fits.writeto(outputV,vel,head,overwrite=True)
fits.writeto(outputLambdaR,lambdaR,head,overwrite=True)
fits.writeto(outputSigma,sigma,head,overwrite=True)

    
print '''\t+---------+\n\t fits saved\n\t+---------+'''     

/home/maccagni/programs/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:34: DeprecationWarning: object of type <type 'numpy.float64'> cannot be safely interpreted as an integer.
/home/maccagni/programs/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:35: DeprecationWarning: object of type <type 'numpy.float64'> cannot be safely interpreted as an integer.


	+---------+
	 fits saved
	+---------+


In [23]:
#----------------------#
# add WCS
#----------------------#

output = resOut+'StarFlux.fits'
outputWcs = resOut+'StarFluxWcs.fits'


data = fits.open(output)[0].data
head = fits.getheader(output)


w = wcs.WCS(naxis=2)

pixelSize = head['PIXSIZE']/3600.

# Set up an "Airy's zenithal" projection
# Vector properties may be set with Python lists, or Numpy arrays
w.wcs.crpix = [crPix1, crPix2]
w.wcs.cdelt = np.array([-pixelSize,pixelSize])
w.wcs.crval = [ra, dec]
w.wcs.ctype = ["RA---SIN", "DEC--SIN"]

header = w.to_header()

del header['RADESYS']
del header['LATPOLE']
del header['LONPOLE']
#w.wcs.set_pv([(2, 1, 45.0)])
header['EQUINOX'] = 2000



hdu = fits.PrimaryHDU(data=data.T,header=header)

hdu.writeto(outputWcs,overwrite=True)

print '''\t+---------+\n\t fits WCS saved\n\t+---------+'''     

	+---------+
	 fits WCS saved
	+---------+


In [24]:
#----------------------#
# add WCS to everything
#----------------------#

dataV = fits.open(outputV)[0].data
dataSigma = fits.open(outputSigma)[0].data
#datah3 = fits.open(outputh3)[0].data
#datah4 = fits.open(outputh4)[0].data
dataLambdaR = fits.open(outputLambdaR)[0].data

outputWcsV = resOut+'StarVelWcs.fits'
outputWcsSigma = resOut+'StarSigmaWcs.fits'
#outputWcsH3 = resOut+'StarHer3Wcs.fits'
#outputWcsH4 = resOut+'StarHer4Wcs.fits'
outputWcsLambdaR = resOut+'StarLambdaRWcs.fits'


hdu = fits.PrimaryHDU(data=dataV.T,header=header)
hdu.writeto(outputWcsV,overwrite=True)

hdu = fits.PrimaryHDU(data=dataSigma.T,header=header)
hdu.writeto(outputWcsSigma,overwrite=True)

#hdu = fits.PrimaryHDU(data=datah3.T,header=header)
#hdu.writeto(outputWcsH3,overwrite=True)

#hdu = fits.PrimaryHDU(data=datah4.T,header=header)
#hdu.writeto(outputWcsH4,overwrite=True)

hdu = fits.PrimaryHDU(data=dataLambdaR.T,header=header)
hdu.writeto(outputWcsLambdaR,overwrite=True)



print '''\t+---------+\n\t fits WCS saved\n\t+---------+'''     

	+---------+
	 fits WCS saved
	+---------+


In [25]:
#----------------------#
# From Fits Table to FITS Image - GANDALF
#----------------------#

diffusion = 1e-5

tab = fits.open(inTable)

head = tab[0].header
headTab = tab[1].header
dataTab = tab[1].data

print dataTab.dtype.names
print dataTab['BIN_ID'].shape
tabGF = fits.open(gandalfTable)

headNamesGF = tabGF[1].header
dataNamesGF = tabGF[1].data

headLinesGF = tabGF[2].header
dataLinesGF = np.array(tabGF[2].data)
tabGF.info()
print dataLinesGF.dtype.names
print dataLinesGF.dtype.names
print dataNamesGF['name']
print dataNamesGF['LINE_ID']


print dataLinesGF['V'].shape


('ID', 'BIN_ID', 'X', 'Y', 'FLUX', 'SNR', 'XBIN', 'YBIN', 'SNRBIN', 'NSPAX')
(96748,)
Filename: /home/maccagni/Projects/MFS/FornaxA/MUSE/cubes/results/FornaxE_5/FornaxE_gandalf_BIN.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       4   ()      
  1  EMISSION_SETUP    1 BinTableHDU     31   22R x 11C   ['D', 'D', '15A', 'D', '15A', '15A', 'D', 'D', 'D', '15A', 'D']   
  2  GANDALF       1 BinTableHDU     21   30628R x 6C   ['22D', '22D', '22D', '22D', '22D', '2D']   
('FLUX', 'AMPL', 'V', 'SIGMA', 'AON', 'EBMV')
('FLUX', 'AMPL', 'V', 'SIGMA', 'AON', 'EBMV')
['HeII' '[NeV]' '[NeV]' '[OII]' '[OII]' '[NeIII]' '[NeIII]' '[OIII]'
 'HeII' '[ArIV]' '[ArIV]' 'Hb' '[OIII]' '[NI]' '[NI]' 'HeI' '[OI]' 'Ha'
 '[NII]' '[SII]' '[SII]' '[ArIII]']
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18. 19. 20. 21.]
(30628, 22)


In [35]:
lineID = 12
outputLine = 'Hb_'
SNR=3.
SNRName='3'

output = resOut+outputLine+'Flux_SNR'+SNRName+'.fits'
outputVGF = resOut+outputLine+'Vel_SNR'+SNRName+'.fits'
outputSigmaGF = resOut+outputLine+'Sigma_SNR'+SNRName+'.fits'


head['CRPIX1'] = crPix1
head['CRPIX2'] = crPix2

xMin = np.min(dataTab['X'])
xMax = np.max(dataTab['X'])

shapeX = (xMax-xMin)/head['PIXSIZE']
yMin = np.min(dataTab['Y'])
yMax = np.max(dataTab['Y'])

shapeY = (yMax-yMin)/head['PIXSIZE']


xAxis = (np.linspace(1, shapeX, shapeX) - head['CRPIX1']) *head['PIXSIZE']
yAxis = (np.linspace(1, shapeY, shapeY) - head['CRPIX2']) *head['PIXSIZE']

xIM, yIM = np.meshgrid(xAxis, yAxis)


data = np.zeros([yAxis.shape[0],xAxis.shape[0]])
data[data==0.0] = np.nan

vel = np.zeros([yAxis.shape[0],xAxis.shape[0]])
vel[vel==0.0] = np.nan

sigma = np.zeros([yAxis.shape[0],xAxis.shape[0]])
sigma[sigma==0.0] = np.nan

#h3 = np.zeros([yAxis.shape[0],xAxis.shape[0]])
#h3[h3==0.0] = np.nan

#h4 = np.zeros([yAxis.shape[0],xAxis.shape[0]])
#h4[h4==0.0] = np.nan

#lambdaR = np.zeros([yAxis.shape[0],xAxis.shape[0]])
#lambdaR[lambdaR==0.0] = np.nan

for i in xrange(0,dataTab['ID'].shape[0]):

    indexX = ((xAxis < (np.round(dataTab['X'][i],1)+diffusion)) & 
              ((np.round(dataTab['X'][i],1)-diffusion) < xAxis))
    indexY = ((yAxis < (np.round(dataTab['Y'][i],1)+diffusion)) & 
              ((np.round(dataTab['Y'][i],1)-diffusion) < yAxis))    
   
    rowGF = dataTab['BIN_ID'][i]
    if  dataLinesGF['AON'][rowGF,lineID] < SNR:
    #print dataLinesGF['AON'][rowGF,lineID]
        vel[indexY,indexX] = np.nan
        sigma[indexY,indexX] = np.nan
        data[indexY,indexX] = np.nan   
    else:
        vel[indexY,indexX] = dataLinesGF['V'][rowGF,lineID]
        sigma[indexY,indexX] = dataLinesGF['SIGMA'][rowGF,lineID]
        data[indexY,indexX] = dataLinesGF['FLUX'][rowGF,lineID]
    
    
#    if np.sum(indexPPXF)>0:
        
#        vel[indexX,indexY] = dataTabPP['V'][indexPPXF]
#        sigma[indexX,indexY] = dataTabPP['SIGMA'][indexPPXF]
#        h3[indexX,indexY] = dataTabPP['H3'][indexPPXF]
#        h4[indexX,indexY] = dataTabPP['H4'][indexPPXF]
#        h4[indexX,indexY] = dataTabPP['H4'][indexPPXF]
#        lambdaR[indexX,indexY] = dataTabPP['LAMBDA_R'][indexPPXF]
        
#    data[indexX,indexY] = dataTab['FLUX'][i]

fits.writeto(output,data,head,overwrite=True)
fits.writeto(outputVGF,vel,head,overwrite=True)
fits.writeto(outputSigmaGF,sigma,head,overwrite=True)

    
print '''\t+---------+\n\t fits saved\n\t+---------+'''     

/home/maccagni/programs/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:24: DeprecationWarning: object of type <type 'numpy.float64'> cannot be safely interpreted as an integer.
/home/maccagni/programs/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:25: DeprecationWarning: object of type <type 'numpy.float64'> cannot be safely interpreted as an integer.


	+---------+
	 fits saved
	+---------+


In [36]:
#----------------------#
# add WCS
#----------------------#

outputLine = 'Hb_'

output = resOut+outputLine+'Flux_SNR'+SNRName+'.fits'
outputWcs = resOut+outputLine+'FluxWcs_SNR'+SNRName+'.fits'


data = fits.open(output)[0].data
head = fits.getheader(output)


w = wcs.WCS(naxis=2)

pixelSize = head['PIXSIZE']/3600.

# Set up an "Airy's zenithal" projection
# Vector properties may be set with Python lists, or Numpy arrays
w.wcs.crpix = [crPix1, crPix2]
w.wcs.cdelt = np.array([-pixelSize,pixelSize])
w.wcs.crval = [ra, dec]
w.wcs.ctype = ["RA---SIN", "DEC--SIN"]

header = w.to_header()

del header['RADESYS']
del header['LATPOLE']
del header['LONPOLE']
#w.wcs.set_pv([(2, 1, 45.0)])
header['EQUINOX'] = 2000

hdu = fits.PrimaryHDU(data=data,header=header)

hdu.writeto(outputWcs,overwrite=True)

print '''\t+---------+\n\t fits WCS saved\n\t+---------+'''     

	+---------+
	 fits WCS saved
	+---------+


In [37]:
#----------------------#
# add WCS to everything
#----------------------#
outputLine='Hb_'

outputVGF = resOut+outputLine+'Vel_SNR'+SNRName+'.fits'
outputSigmaGF = resOut+outputLine+'Sigma_SNR'+SNRName+'.fits'

dataV = fits.open(outputVGF)[0].data
dataSigma = fits.open(outputSigmaGF)[0].data
#datah3 = fits.open(outputh3)[0].data
#datah4 = fits.open(outputh4)[0].data
#dataLamdaR = fits.open(outputLambdaR)[0].data

outputWcsV = resOut+outputLine+'VelWcs_SNR'+SNRName+'.fits'
outputWcsSigma = resOut+outputLine+'StarSigmaWcs_SNR'+SNRName+'.fits'
#outputWcsH3 = resOut+'StarHer3Wcs.fits'
#outputWcsH4 = resOut+'StarHer4Wcs.fits'
#outputWcsLambdaR = resOut+'StarLambdaRWcs.fits'


hdu = fits.PrimaryHDU(data=dataV,header=header)
hdu.writeto(outputWcsV,overwrite=True)

hdu = fits.PrimaryHDU(data=dataSigma,header=header)
hdu.writeto(outputWcsSigma,overwrite=True)

#hdu = fits.PrimaryHDU(data=datah3.T,header=header)
#hdu.writeto(outputWcsH3,overwrite=True)

#hdu = fits.PrimaryHDU(data=datah4.T,header=header)
#hdu.writeto(outputWcsH4,overwrite=True)

#hdu = fits.PrimaryHDU(data=dataLambdaR.T,header=header)
#hdu.writeto(outputWcsLambdaR,overwrite=True)



print '''\t+---------+\n\t fits WCS saved\n\t+---------+'''     

	+---------+
	 fits WCS saved
	+---------+
